In [1]:
import numpy as np
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import os
global image_paths, target_size
basedir = '../Datasets/cartoon_set'
grayscale = True
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

In [2]:
import numpy as np
# from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
import cv2
import dlib

# PATH TO ALL IMAGES

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')


# how to find frontal human faces in an image using 68 landmarks.  These are points on the face such as the corners of the mouth, along the eyebrows, on the eyes, and so forth.

# The face detector we use is made using the classic Histogram of Oriented
# Gradients (HOG) feature combined with a linear classifier, an image pyramid,
# and sliding window detection scheme.  The pose estimator was created by
# using dlib's implementation of the paper:
# One Millisecond Face Alignment with an Ensemble of Regression Trees by
# Vahid Kazemi and Josephine Sullivan, CVPR 2014
# and was trained on the iBUG 300-W face landmark dataset (see https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/):
#     C. Sagonas, E. Antonakos, G, Tzimiropoulos, S. Zafeiriou, M. Pantic.
#     300 faces In-the-wild challenge: Database and results.
#     Image and Vision Computing (IMAVIS), Special Issue on Facial Landmark Localisation "In-The-Wild". 2016.


def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')
    
    if grayscale:
        gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        gray = gray.astype('uint8')
    else:
        gray = resized_image
        
    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extract the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    gender_labels = {line.split('\t')[0] : int(line.split('\t')[2]) for line in lines[1:]}
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            #windows: file_name= img_path.split('.')[2].split('\\')[-1]
            #mac: file_name= img_path.split('.')[2].split('/')[-1]
            file_name= img_path.split('.')[2].split('\\')[-1]

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(gender_labels[file_name])

    landmark_features = np.array(all_features)
    gender_labels = np.array(all_labels)
    return landmark_features, gender_labels

In [3]:
def printOutput(m):
    print("accuracy", accuracy_score(te_Y, m))
    print("precision", precision_score(te_Y, m, average = None))
    print("recall", recall_score(te_Y, m, average = None))

In [5]:
tr_X, y = extract_features_labels()
tr_Y = np.array([y, -(y - 1)]).T
basedir = '../Datasets/cartoon_set_test'
images_dir = os.path.join(basedir,'img')
te_X, y = extract_features_labels()
te_Y = np.array([y, -(y - 1)]).T
tr_Y = list(zip(*tr_Y))[0]
te_Y = list(zip(*te_Y))[0]

In [55]:
print(len(tr_Y))
print(len(te_Y))
count = [0,0,0,0,0]
for ele in tr_Y:
    count[int(ele)] += 1
print(count)

8393
2088
[1657, 1600, 1768, 1712, 1656]


In [ ]:
"""
training: 8194 testing: 2041
num of sample for each class: [1608, 1573, 1744, 1652, 1617]

record of modle output
svm:
accuracy 0.7143557079862812
precision [0.72885572 0.67705382 0.73362445 0.68459658 0.7398568 ]
recall [0.75710594 0.60201511 0.75675676 0.67961165 0.77306733]

svm one vs one:
accuracy 0.7192552670259676
precision [0.7578125  0.69164265 0.73903509 0.66981132 0.73488372]
recall [0.75193798 0.60453401 0.75900901 0.68932039 0.78802993]

knn: k = 17
accuracy 0.5605095541401274
precision [0.55525606 0.39716312 0.59607843 0.50502513 0.78171091]
recall [0.53229974 0.4231738  0.68468468 0.48786408 0.66084788]

adaboost: estimators = 75
accuracy 0.6065654091131798
precision [0.67279412 0.49873418 0.62049336 0.59898477 0.65121413]
recall [0.47286822 0.49622166 0.73648649 0.57281553 0.73566085]

svm one vs rest with only face features:
accuracy 0.7089661930426262
precision [0.74111675 0.70731707 0.72943723 0.64814815 0.72      ]
recall [0.75452196 0.58438287 0.75900901 0.67961165 0.76309227]

SVM one vs one with only face features:
accuracy 0.7104360607545321
precision [0.75452196 0.71384615 0.72943723 0.64155251 0.71794872]
recall [0.75452196 0.58438287 0.75900901 0.68203883 0.7680798 ]

KNN with only face features: k = 7
accuracy 0.6746692797648212
precision [0.68632708 0.56812339 0.71428571 0.63245823 0.76822917]
recall [0.66149871 0.55667506 0.76576577 0.64320388 0.73566085]

adaboos wih face feaures: 100
accuracy 0.6134247917687408
precision [0.64220183 0.5104712  0.62666667 0.57211538 0.71611253]
recall [0.54263566 0.49118388 0.74099099 0.5776699  0.69825436]

svm with face and eyebrow features:
accuracy 0.7177853993140617
precision [0.73846154 0.70724638 0.7311828  0.67625899 0.73349057]
recall [0.74418605 0.61460957 0.76576577 0.68446602 0.7755611 ]

svm one vs one with face and eyebrow features:
accuracy 0.7192552670259676
precision [0.75328084 0.70689655 0.73275862 0.67061611 0.73239437]
recall [0.74160207 0.61964736 0.76576577 0.6868932  0.77805486]

KNN with face and eyebrow features: K = 10
accuracy 0.5889269965703087
precision [0.60695187 0.42716049 0.62698413 0.52631579 0.76880223]
recall [0.58656331 0.43576826 0.71171171 0.50970874 0.6882793 ]

ada with face and eyebrow features: 100
accuracy 0.6109750122488976
precision [0.64356436 0.50125313 0.63261297 0.58595642 0.69064748]
recall [0.50387597 0.50377834 0.72522523 0.58737864 0.71820449]

8393
2088
[1657, 1600, 1768, 1712, 1656]

svm one vs rest without grayscale
accuracy 0.7389846743295019
precision [0.72330097 0.7112462  0.75298805 0.75       0.74831461]
recall [0.76606684 0.58208955 0.83259912 0.69444444 0.81021898]

svm one vs one without grayscale
accuracy 0.7432950191570882
precision [0.74494949 0.72560976 0.75903614 0.74038462 0.74      ]
recall [0.75835476 0.5920398  0.83259912 0.71296296 0.81021898]

svm one vs rest with only face features without grayscale
accuracy 0.7413793103448276
precision [0.70862471 0.75308642 0.76844262 0.74757282 0.72873563]
recall [0.781491   0.60696517 0.82599119 0.71296296 0.77128954]

svm one vs one with only face features without grayscale
accuracy 0.7437739463601533
precision [0.72209026 0.75548589 0.77207392 0.7452381  0.72335601]
recall [0.781491   0.59950249 0.82819383 0.72453704 0.77615572]

svm one vs rest with face and eyebrow features without grayscale:
accuracy 0.7394636015325671
precision [0.71100917 0.75       0.76337449 0.74271845 0.73023256]
recall [0.79691517 0.60447761 0.81718062 0.70833333 0.76399027]

svm one vs one with face and eyebrow features without grayscale:
accuracy 0.7404214559386973
precision [0.71962617 0.74769231 0.76763485 0.73396675 0.73148148]
recall [0.79177378 0.60447761 0.81497797 0.71527778 0.76885645]

"""

In [7]:
s = "0.72885572 0.67705382 0.73362445 0.68459658 0.7398568"
temp = s.split(" ")
temp = list(map(float, temp))
sum(temp)/len(temp)

0.712797474

In [7]:
svmovr = svm.SVC(kernel = "linear").fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
svmovr_pred = svmovr.predict(te_X.reshape((len(te_X), 68*2)))

In [8]:
printOutput(svmovr_pred)

accuracy 0.7143557079862812
precision [0.72885572 0.67705382 0.73362445 0.68459658 0.7398568 ]
recall [0.75710594 0.60201511 0.75675676 0.67961165 0.77306733]


In [9]:
svmovo = OneVsOneClassifier(svm.SVC(kernel = "linear")).fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
svmovo_pred = svmovo.predict(te_X.reshape((len(te_X), 68*2)))

In [10]:
printOutput(svmovo_pred)

accuracy 0.7192552670259676
precision [0.7578125  0.69164265 0.73903509 0.66981132 0.73488372]
recall [0.75193798 0.60453401 0.75900901 0.68932039 0.78802993]


In [12]:
aknn = []
#sqr(8194) = 91
for i in range(1, 92):
    classifier = KNeighborsClassifier(n_neighbors=k, algorithm = 'brute')
    scores = cross_val_score(classifier, tr_X.reshape((len(tr_X), 68*2)), tr_Y, cv = 5)
    aknn.append([scores.mean(), scores.std()])

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

...

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

In [15]:
print(len(aknn))

91


In [16]:
pd.DataFrame(aknn).to_csv("knnoutput.csv")

In [23]:
knn = KNeighborsClassifier(n_neighbors=17, algorithm = 'brute')
knn.fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
knn_pred = knn.predict(te_X.reshape((len(te_X), 68*2)))
printOutput(knn_pred)

accuracy 0.5605095541401274
precision [0.55525606 0.39716312 0.59607843 0.50502513 0.78171091]
recall [0.53229974 0.4231738  0.68468468 0.48786408 0.66084788]


D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [20]:
aada = []

for i in range(1, 200):    
    classifier = AdaBoostClassifier(n_estimators=i, random_state=0)
    scores = cross_val_score(classifier, tr_X.reshape((len(tr_X), 68*2)), tr_Y, cv = 5)
    aada.append([scores.mean(), scores.std()])

In [22]:
print(len(aada))

199


In [24]:
pd.DataFrame(aada).to_csv("adaoutput.csv")

In [25]:
clf = AdaBoostClassifier(n_estimators=75, random_state = 0)
clf.fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
clf_pred = clf.predict(te_X.reshape((len(te_X), 68*2)))

In [26]:
printOutput(clf_pred)

accuracy 0.6065654091131798
precision [0.67279412 0.49873418 0.62049336 0.59898477 0.65121413]
recall [0.47286822 0.49622166 0.73648649 0.57281553 0.73566085]


In [5]:
#changing dataset with specific features needed
def selectingFeatures(x, y):
    training = []
    for ele in tr_X.reshape((len(tr_X), 68*2)):
        training.append(ele[x:y])
    testing = []
    for ele in te_X.reshape((len(te_X), 68*2)):
        testing.append(ele[x:y])
    return training, testing

In [29]:
len(tx)

8194

In [42]:
svmovrt = svm.SVC(kernel = "linear").fit(tx, tr_Y)
svmovrt_pred = svmovrt.predict(tex)

In [43]:
printOutput(svmovrt_pred)

accuracy 0.7177853993140617
precision [0.73846154 0.70724638 0.7311828  0.67625899 0.73349057]
recall [0.74418605 0.61460957 0.76576577 0.68446602 0.7755611 ]


In [44]:
svmovot = OneVsOneClassifier(svm.SVC(kernel = "linear")).fit(tx, tr_Y)
svmovot_pred = svmovot.predict(tex)
printOutput(svmovot_pred)

accuracy 0.7192552670259676
precision [0.75328084 0.70689655 0.73275862 0.67061611 0.73239437]
recall [0.74160207 0.61964736 0.76576577 0.6868932  0.77805486]


In [45]:
aknnt = []
for i in range(1, 91):
    classifier = KNeighborsClassifier(n_neighbors=i, algorithm = 'brute')
    
    scores = cross_val_score(classifier, tx, tr_Y, cv = 5)

    aknnt.append([scores.mean(), scores.std()])

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

...

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

In [46]:
print(len(aknnt))

90


In [47]:
pd.DataFrame(aknnt).to_csv("knnttoutput.csv")

In [48]:
knnt = KNeighborsClassifier(n_neighbors=10, algorithm = 'brute')
knnt.fit(tx, tr_Y)
knnt_pred = knnt.predict(tex)
printOutput(knnt_pred)

accuracy 0.5889269965703087
precision [0.60695187 0.42716049 0.62698413 0.52631579 0.76880223]
recall [0.58656331 0.43576826 0.71171171 0.50970874 0.6882793 ]


D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [49]:
clf1 = AdaBoostClassifier(n_estimators=100, random_state=0)
clf1.fit(tx, tr_Y)
clf1_pred = clf1.predict(tex)

In [50]:
printOutput(clf1_pred)

accuracy 0.6109750122488976
precision [0.64356436 0.50125313 0.63261297 0.58595642 0.69064748]
recall [0.50387597 0.50377834 0.72522523 0.58737864 0.71820449]


In [4]:
#preprocssing images without grayscale
grayscale = False
tr_X, y = extract_features_labels()
tr_Y = np.array([y, -(y - 1)]).T
basedir = '../Datasets/cartoon_set_test'
images_dir = os.path.join(basedir,'img')
te_X, y = extract_features_labels()
te_Y = np.array([y, -(y - 1)]).T
tr_Y = list(zip(*tr_Y))[0]
te_Y = list(zip(*te_Y))[0]

In [20]:
svmovr = svm.SVC(kernel = "linear").fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
svmovr_pred = svmovr.predict(te_X.reshape((len(te_X), 68*2)))
printOutput(svmovr_pred)

accuracy 0.7389846743295019
precision [0.72330097 0.7112462  0.75298805 0.75       0.74831461]
recall [0.76606684 0.58208955 0.83259912 0.69444444 0.81021898]


In [21]:
svmovo = OneVsOneClassifier(svm.SVC(kernel = "linear")).fit(tr_X.reshape((len(tr_X), 68*2)), tr_Y)
svmovo_pred = svmovo.predict(te_X.reshape((len(te_X), 68*2)))
printOutput(svmovo_pred)

accuracy 0.7432950191570882
precision [0.74494949 0.72560976 0.75903614 0.74038462 0.74      ]
recall [0.75835476 0.5920398  0.83259912 0.71296296 0.81021898]


In [16]:
tx, tex = selectingFeatures(0, 34)

In [22]:
svmovrt = svm.SVC(kernel = "linear").fit(tx, tr_Y)
svmovrt_pred = svmovrt.predict(tex)
printOutput(svmovrt_pred)

accuracy 0.7413793103448276
precision [0.70862471 0.75308642 0.76844262 0.74757282 0.72873563]
recall [0.781491   0.60696517 0.82599119 0.71296296 0.77128954]


In [18]:
svmovot = OneVsOneClassifier(svm.SVC(kernel = "linear")).fit(tx, tr_Y)
svmovot_pred = svmovot.predict(tex)
printOutput(svmovot_pred)

accuracy 0.7437739463601533
precision [0.72209026 0.75548589 0.77207392 0.7452381  0.72335601]
recall [0.781491   0.59950249 0.82819383 0.72453704 0.77615572]
